In [ ]:
!pip install cv2
!pip install pytesseract
!pip install numpy

In [4]:
from PIL import Image
import cv2
import numpy as np
import os
import matplotlib as plt
import math
import scipy.ndimage

In [13]:
def remove_noise(image_path, save_path):
    # Open the image using Pillow
    image = Image.open(image_path)

    # Convert the image to grayscale
    gray_image = image.convert('L')

    # Convert PIL image to numpy array
    np_image = np.array(gray_image)

    # Apply binary thresholding
    _, binary_image = cv2.threshold(np_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Apply morphological operations
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=1)  # Increase iterations for more noise removal

    # Apply Gaussian blur
    denoised_image = cv2.GaussianBlur(opening, (3, 3), 0)

    # Apply median blur to further reduce noise, particularly small circles
    denoised_image = cv2.medianBlur(denoised_image, 3)

    # Save the denoised image
    denoised_image_pil = Image.fromarray(denoised_image)
    denoised_image_pil.save(save_path)


In [26]:
# Generate denoised images
folder = 'four_cap_36'
output_folder = 'denoised_images'

# Get list of all files in the folder
file_list = os.listdir(folder)

# Iterate through the first 10 images in the folder
for i, filename in enumerate(file_list):
    # Check if the file is an image (you can add more image extensions if needed)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Construct the full path to the image
        image_path = os.path.join(folder, filename)
        
        # Save path for the denoised image
        # original image -> denoised image (now named after its label)
        label = filename.split('-')[0]
        save_filename = f'{label}_{i}_denoised.png' # There is a possibility that the images might have the same label -- TODO: FIX IT later
        save_path = os.path.join(output_folder, save_filename)
        
        # Call the remove_noise function
        remove_noise(image_path, save_path)


In [41]:
# trying to resize the image
def resize_image(image, new_width, new_height):
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height))
    return resized_image

def save_contours_as_images(image_path, output_directory):
    # Load the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Threshold the image to obtain binary image
    _, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on x-coordinate
    contours = sorted(contours, key=lambda contour: cv2.boundingRect(contour)[0])

    # Create output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    for i, contour in enumerate(contours):
        # Get bounding box for each contour
        x, y, w, h = cv2.boundingRect(contour)

        # Check if contour is too small (possibly noise)
        if w > 5 and h > 5:
            # Add some padding around the character bounding box
            padding = 10
            x_padding = max(0, x - padding)
            y_padding = max(0, y - padding)
            w_padding = min(image.shape[1], w + 2 * padding)
            h_padding = min(image.shape[0], h + 2 * padding)

            # Create a black canvas with padded dimensions
            padded_image = np.zeros((h_padding, w_padding), dtype=np.uint8)

            # Calculate coordinates to place the character in the center
            x_offset = (w_padding - w) // 2
            y_offset = (h_padding - h) // 2

            # Copy the character region from the original image to padded image
            padded_image[y_offset:y_offset+h, x_offset:x_offset+w] = image[y:y+h, x:x+w]

            # Resize the padded image
            resized_image = resize_image(padded_image, 100, 100)

            # Save the resized image as a separate image
            # TODO: CHANGE NAME TO LABEL (include it in the for loop above)
            label = image_path.split('/')[0].split('.')[0].split("\\")[1]
            print(label)
            char_labels = [char_label for char_label in label]
            for char in char_labels:
                character_filename = os.path.join(output_directory, f'character_{char}.png')
                cv2.imwrite(character_filename, resized_image)


In [42]:
# Folder path containing the images
folder_path = 'denoised_images'

# Output directory for saved contour images
output_directory = 'cropped_characters'

# Get list of all files in the folder
file_list = os.listdir(folder_path)

# Iterate through the first 10 images in the folder
for i, filename in enumerate(file_list):
    if i >= 1:
        break
    # Check if the file is an image (you can add more image extensions if needed)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Construct the full path to the image
        image_path = os.path.join(folder_path, filename)
        
        # Call the save_contours_as_images function
        save_contours_as_images(image_path, output_directory)
        
        print(f'Contours saved for image: {filename}')

11P5
11P5
11P5
11P5
Contours saved for image: 11P5.png
